In [1]:
# Exploratory Notebook July 14 2020
import pandas as pd
import numpy as np
import altair as alt

In this notebook, we will explore the physician compare datasets, taken from [medicare.gov](https://data.medicare.gov/data/physician-compare). Info about physican compare can be found [here](https://www.medicare.gov/physiciancompare/#about/improvinghealthcarequality). Let's start by looking at the Measures dataset and settle on what these datasets can offer us. Physican Compare is an extension of medicare that helps US citizens connect with physicians, and in some cases, compare performances between individuals and groups.

In [2]:
measures = pd.read_csv("data/Physician_Compare_2017_Group_Public_Reporting_-_Measures.csv")

In [3]:
measures.head()

,Organization legal name or 'doing business as' name,Group PAC ID,State,ACO PC ID 1,ACO Name 1,ACO PC ID 2,ACO Name 2,Measure Code,Measure Title,Inverse Measure,Attestation Value,Measure Performance Rate,Denominator Count,Star Value,Five Star Benchmark,Collection Type,Reported on PC Live Site
0,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_EP_1,e-Prescribing,N,NaN,88.0,16076.0,NaN,NaN,ATT,N
1,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_HIE_1,Health Information Exchange,N,NaN,67.0,2581.0,NaN,NaN,ATT,N
2,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_MR_1,Medication Reconciliation,N,NaN,95.0,1676.0,NaN,NaN,ATT,N
3,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PEA_1,Provide Patient Access,N,NaN,78.0,5918.0,NaN,NaN,ATT,N
4,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PEA_2,"View, Download, or Transmit (VDT)",N,NaN,3.0,5878.0,NaN,NaN,ATT,N


In [4]:
measures.shape

(73321, 17)

In [5]:
measures.isnull().sum()

Organization legal name or 'doing business as' name        0
Group PAC ID                                               0
State                                                      0
ACO PC ID 1                                            49426
ACO Name 1                                             49426
ACO PC ID 2                                            72878
ACO Name 2                                             72878
Measure Code                                               0
Measure Title                                              0
Inverse Measure                                            0
Attestation Value                                      52828
Measure Performance Rate                               20493
Denominator Count                                      20493
Star Value                                             70125
Five Star Benchmark                                    70125
Collection Type                                        20493
Reported on PC Live Site

Several columns have > 90%  missing values, which is kind of a problem. These are probably unusable columns. Some columns have no missing values though, so there is still a lot of potential for us to use this dataset. The next state of business is to understand what each column means.

In [6]:
# reformat column names to be more appropriate
columns = ["org_name", "group_PAC_ID", "state", "ACO_PC_ID_1", "ACO_name_1",
           "ACO_PC_ID_2", "ACO_name_2", "measure_code", "measure_title", 
          "inverse_measure", "attestation_value", "measure_performance_rate", "denominator_count", 
          "star_value", "five_star_benchmark", "collection_type", "from_PC_livesite"]

In [7]:
measures.columns = columns

A few initial observations and questions:
- group most likely refers to "hospital group", where group_PAC_ID may be unique to each org
    - **confirm group_ID as identifying key for org group**
    - Ans: confirmed
- **what do ACO_PC_ID and ACO_name mean?**
- measure code and title seems to deal with how the performances are calculated for each hospital
    - **do all hospitals have the same set of measures?**
- **what is performance rate and what's its range?**
    - I think denominator count contributes to either the performance rate or the star-value calculation
    - Ans: performance rate is in the form of a percentage, ranging from 0-100
- star value goes from 1-7 (confirm?) and is based on...
- collection type is a little vague
    - **what is collection type?**
    - Ans: 4 categories of collection types: Web-interface (WI), Registry (REG),Qualified Clinical Data Registry (QCDR), CMS Web Attestation (ATT)
- from_PC_livesite seems to be whether it comes from the physican compare live site or not. 
    - **is there any value in keeping this column?**

Some questions were simple to answer by looking at the data dictionary, but there are a few questions that will require some deeper investigation.

## What do ACO_PC_ID and ACO_name mean?

ACO stands for Accountable Care Organization and seems to be related to different measure types. For example, ACO-5 is the following measure: CAHPS: Health Promotion and Education, or in other words, Health promotion and education. In the context of the data, however, it may have a different meaning.ACO's are organizations that were founded to keep healthcare practices accountable, by paying based on efficiency of care, rather than amount of care. There are about [744](https://www.healthaffairs.org/do/10.1377/hblog20150331.045829/full/) organizations in America. Approximately 472 are covered in this dataset. 

In [9]:
len(measures.ACO_PC_ID_1.unique())

472

From here out, we will simply treat the ACO_PC_ID as a unique identifier for these ACO organizations. 

## Do all Hospitals have the same set of measures

In [10]:
measures[measures["org_name"]== "BRIDGTON HOSPITAL"]

,org_name,group_PAC_ID,state,ACO_PC_ID_1,ACO_name_1,ACO_PC_ID_2,ACO_name_2,measure_code,measure_title,inverse_measure,attestation_value,measure_performance_rate,denominator_count,star_value,five_star_benchmark,collection_type,from_PC_livesite
0,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_EP_1,e-Prescribing,N,NaN,88.0,16076.0,NaN,NaN,ATT,N
1,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_HIE_1,Health Information Exchange,N,NaN,67.0,2581.0,NaN,NaN,ATT,N
2,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_MR_1,Medication Reconciliation,N,NaN,95.0,1676.0,NaN,NaN,ATT,N
3,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PEA_1,Provide Patient Access,N,NaN,78.0,5918.0,NaN,NaN,ATT,N
4,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PEA_2,"View, Download, or Transmit (VDT)",N,NaN,3.0,5878.0,NaN,NaN,ATT,N
5,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PHCDRR_1,Immunization Registry Reporting,N,Y,NaN,NaN,NaN,NaN,NaN,N
6,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PHCDRR_2,Syndromic Surveillance Reporting,N,Y,NaN,NaN,NaN,NaN,NaN,N
7,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PPHI_1,Security Risk Analysis,N,Y,NaN,NaN,NaN,NaN,NaN,N
8,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_PSE_1,Patient-Specific Education,N,NaN,38.0,7115.0,NaN,NaN,ATT,N
9,BRIDGTON HOSPITAL,8123919099,ME,A1080,Central Maine ACO,NaN,NaN,ACI_GRP_TRANS_SM_1,Secure Messaging,N,NaN,3.0,5878.0,NaN,NaN,ATT,N


In [11]:
measure_count = measures.groupby("org_name").agg({"measure_title":"count"}).reset_index()

In [12]:
measure_count = measure_count.sort_values(by ="measure_title")

In [13]:
measure_count

,org_name,measure_title
10138,"SALT LAKE RETINA, LLC",1
10741,SOUTHERN FAMILY MEDICAL CENTER PC,1
9429,RADIOLOGY ASSOCIATES OF SEO INC,1
11838,TURLOCK EYE PHYSICIANS MEDICAL GROUP,1
11840,TURNING POINT CARE CENTER LLC,1
...,...,...
12008,UNIVERSITY OF MIAMI,47
4968,HEALTHTEXAS PROVIDER NETWORK,48
4768,GREENWOOD LEFLORE HOSPITAL,77
10049,RUSH UNIVERSITY MEDICAL CENTER,78


In [14]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [15]:
alt.Chart(measure_count).mark_bar().encode(
    alt.X("measure_title", bin = True),
    alt.Y("count()")
)

alt.Chart(...)

There definitely is a disparency in the number of measures for each organization. One in particular, "Regents of hte university of Cali", had 160 different measure titles.

In [16]:
measures[measures["org_name"] == "REGENTS OF THE UNIVERSITY OF CALIFORNIA"]

,org_name,group_PAC_ID,state,ACO_PC_ID_1,ACO_name_1,ACO_PC_ID_2,ACO_name_2,measure_code,measure_title,inverse_measure,attestation_value,measure_performance_rate,denominator_count,star_value,five_star_benchmark,collection_type,from_PC_livesite
13252,REGENTS OF THE UNIVERSITY OF CALIFORNIA,3779689518,CA,NaN,NaN,NaN,NaN,ACI_GRP_TRANS_EP_1,e-Prescribing,N,NaN,82.0,24081.0,NaN,NaN,ATT,N
13253,REGENTS OF THE UNIVERSITY OF CALIFORNIA,3779689518,CA,NaN,NaN,NaN,NaN,ACI_GRP_TRANS_HIE_1,Health Information Exchange,N,NaN,7.0,263.0,NaN,NaN,ATT,N
13254,REGENTS OF THE UNIVERSITY OF CALIFORNIA,3779689518,CA,NaN,NaN,NaN,NaN,ACI_GRP_TRANS_MR_1,Medication Reconciliation,N,NaN,99.0,24808.0,NaN,NaN,ATT,N
13255,REGENTS OF THE UNIVERSITY OF CALIFORNIA,3779689518,CA,NaN,NaN,NaN,NaN,ACI_GRP_TRANS_PEA_1,Provide Patient Access,N,NaN,100.0,24808.0,NaN,NaN,ATT,N
13256,REGENTS OF THE UNIVERSITY OF CALIFORNIA,3779689518,CA,NaN,NaN,NaN,NaN,ACI_GRP_TRANS_PEA_2,"View, Download, or Transmit (VDT)",N,NaN,8.0,24808.0,NaN,NaN,ATT,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73055,REGENTS OF THE UNIVERSITY OF CALIFORNIA,7517869605,CA,NaN,NaN,NaN,NaN,MIPS_GRP_130_overall,Documentation of Current Medications in the Me...,N,NaN,95.0,568.0,NaN,NaN,REG,N
73056,REGENTS OF THE UNIVERSITY OF CALIFORNIA,7517869605,CA,NaN,NaN,NaN,NaN,MIPS_GRP_131_overall,Pain Assessment and Follow-Up,N,NaN,53.0,826.0,NaN,NaN,REG,N
73057,REGENTS OF THE UNIVERSITY OF CALIFORNIA,7517869605,CA,NaN,NaN,NaN,NaN,MIPS_GRP_331_overall,Adult Sinusitis: Antibiotic Prescribed for Acu...,Y,NaN,75.0,59.0,NaN,NaN,REG,N
73098,REGENTS OF THE UNIVERSITY OF CALIFORNIA,3870407455,CA,A1472,Regents of the University of California,NaN,NaN,ACI_GRP_TRANS_PHCDRR_1,Immunization Registry Reporting,N,Y,NaN,NaN,NaN,NaN,NaN,N


# The Star Value

The star value was a implemented to give doctors, in a field that is on the more critized side, a chance for a fair comparison. For example, maybe the best score is 80% for a certain procedure. If a patient sees 80%, they may not be satisfied by the score and won't select the doctor, even though that doctor is performing the best in the field. 

In [17]:
measures[measures['star_value'].isnull() == False]

,org_name,group_PAC_ID,state,ACO_PC_ID_1,ACO_name_1,ACO_PC_ID_2,ACO_name_2,measure_code,measure_title,inverse_measure,attestation_value,measure_performance_rate,denominator_count,star_value,five_star_benchmark,collection_type,from_PC_livesite
69,EASTERN MAINE MEDICAL CENTER,2062315161,ME,V131,Beacon Health LLC,NaN,NaN,MIPS_GRP_110_overall,Preventive Care and Screening: Influenza Immun...,N,NaN,92.0,209.0,4.0,94.0,WI,Y
70,EASTERN MAINE MEDICAL CENTER,2062315161,ME,V131,Beacon Health LLC,NaN,NaN,MIPS_GRP_111_overall,Pneumococcal Vaccination Status for Older Adults,N,NaN,87.0,249.0,4.0,94.0,WI,Y
71,EASTERN MAINE MEDICAL CENTER,2062315161,ME,V131,Beacon Health LLC,NaN,NaN,MIPS_GRP_113_overall,Colorectal Cancer Screening,N,NaN,68.0,249.0,4.0,84.0,WI,Y
72,EASTERN MAINE MEDICAL CENTER,2062315161,ME,V131,Beacon Health LLC,NaN,NaN,MIPS_GRP_134_overall,Preventive Care and Screening: Screening for C...,N,NaN,87.0,291.0,4.0,91.0,WI,Y
73,EASTERN MAINE MEDICAL CENTER,2062315161,ME,V131,Beacon Health LLC,NaN,NaN,MIPS_GRP_226_overall,Preventive Care and Screening: Tobacco Use: Sc...,N,NaN,96.0,250.0,4.0,100.0,WI,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73295,KAUAI MEDICAL CLINIC,5092628479,HI,NaN,NaN,NaN,NaN,MIPS_GRP_113_overall,Colorectal Cancer Screening,N,NaN,72.0,605.0,4.0,84.0,WI,Y
73296,KAUAI MEDICAL CLINIC,5092628479,HI,NaN,NaN,NaN,NaN,MIPS_GRP_134_overall,Preventive Care and Screening: Screening for C...,N,NaN,64.0,610.0,3.0,91.0,WI,Y
73297,KAUAI MEDICAL CLINIC,5092628479,HI,NaN,NaN,NaN,NaN,MIPS_GRP_226_overall,Preventive Care and Screening: Tobacco Use: Sc...,N,NaN,94.0,611.0,4.0,100.0,WI,Y
73298,KAUAI MEDICAL CLINIC,5092628479,HI,NaN,NaN,NaN,NaN,MIPS_GRP_318_overall,Falls: Screening for Future Fall Risk,N,NaN,80.0,605.0,4.0,99.0,WI,Y


According to the star_ratings_cutoff doc in this folder, only a subset of measures actually get a star rating. These are chosen with user resonation and statistical robustness in mind.

In [18]:
# create a measures code book
measure_dict = dict()
def create_dict(code, title, code_dict):
    code_dict[code] = title
    return 

In [19]:
measure_subset = measures[["measure_code", "measure_title"]].copy()

In [20]:
measure_subset.drop_duplicates().apply(lambda x: create_dict(x['measure_code'], x['measure_title'], measure_dict), axis = 1)
print("completed")

completed


In [21]:
ACO_subset = measures[['ACO_PC_ID_1', "ACO_name_1"]].copy()

In [22]:
ACO_dict = dict()
ACO_subset.dropna().drop_duplicates().apply(lambda x: create_dict(x['ACO_PC_ID_1'], x['ACO_name_1'], ACO_dict), axis = 1)
print("complete")

complete


In [23]:
#ACO_dict

In [24]:
# note to self, it would be cool to map out the ACO territories and see how their performances compare

## Data Cleaning

Remove rows and columns:
1. remove missing performance rows
2. investigate ACO 1 and ACO 2

In [25]:
measures_inter = measures.dropna(subset = ["collection_type"])
# drop rows where there was no collection data; this aligned perfectly with denominator count and 
# performance rate

In [26]:
measures_inter.isnull().sum()

org_name                        0
group_PAC_ID                    0
state                           0
ACO_PC_ID_1                 36300
ACO_name_1                  36300
ACO_PC_ID_2                 52531
ACO_name_2                  52531
measure_code                    0
measure_title                   0
inverse_measure                 0
attestation_value           52828
measure_performance_rate        0
denominator_count               0
star_value                  49632
five_star_benchmark         49632
collection_type                 0
from_PC_livesite                0
dtype: int64

In [27]:
measures_inter[(measures_inter["ACO_PC_ID_1"].isnull() == False) & (measures_inter["ACO_PC_ID_2"].isnull() == False)]

,org_name,group_PAC_ID,state,ACO_PC_ID_1,ACO_name_1,ACO_PC_ID_2,ACO_name_2,measure_code,measure_title,inverse_measure,attestation_value,measure_performance_rate,denominator_count,star_value,five_star_benchmark,collection_type,from_PC_livesite
124,MOUNT DESERT ISLAND HOSPITAL,941250963,ME,A3151,"Beacon Rural Health, LLC",V131,Beacon Health LLC,ACI_GRP_TRANS_EP_1,e-Prescribing,N,NaN,15.0,155.0,NaN,NaN,ATT,N
125,MOUNT DESERT ISLAND HOSPITAL,941250963,ME,A3151,"Beacon Rural Health, LLC",V131,Beacon Health LLC,ACI_GRP_TRANS_HIE_1,Health Information Exchange,N,NaN,54.0,194.0,NaN,NaN,ATT,N
126,MOUNT DESERT ISLAND HOSPITAL,941250963,ME,A3151,"Beacon Rural Health, LLC",V131,Beacon Health LLC,ACI_GRP_TRANS_MR_1,Medication Reconciliation,N,NaN,86.0,309.0,NaN,NaN,ATT,N
127,MOUNT DESERT ISLAND HOSPITAL,941250963,ME,A3151,"Beacon Rural Health, LLC",V131,Beacon Health LLC,ACI_GRP_TRANS_PEA_1,Provide Patient Access,N,NaN,7.0,244.0,NaN,NaN,ATT,N
128,MOUNT DESERT ISLAND HOSPITAL,941250963,ME,A3151,"Beacon Rural Health, LLC",V131,Beacon Health LLC,ACI_GRP_TRANS_PEA_2,"View, Download, or Transmit (VDT)",N,NaN,68.0,244.0,NaN,NaN,ATT,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72473,J ROBERT WEST MD INC,8729990973,CA,V233,Heritage California ACO,V235,"Arizona Care Network - Next, LLC",MIPS_GRP_265_overall,Biopsy Follow-Up,N,NaN,0.0,245.0,NaN,NaN,REG,N
72474,J ROBERT WEST MD INC,8729990973,CA,V233,Heritage California ACO,V235,"Arizona Care Network - Next, LLC",MIPS_GRP_402_overall,Tobacco Use and Help with Quitting Among Adole...,N,NaN,93.0,149.0,NaN,NaN,REG,N
72475,J ROBERT WEST MD INC,8729990973,CA,V233,Heritage California ACO,V235,"Arizona Care Network - Next, LLC",MIPS_GRP_431_overall,Preventive Care and Screening: Unhealthy Alcoh...,N,NaN,33.0,1160.0,NaN,NaN,REG,N
72971,"PREMIER NEPHROLOGY MEDICAL GROUP, INC.",5193709947,CA,V233,Heritage California ACO,V239,"APA ACO, Inc.",MIPS_GRP_119_overall,Diabetes: Medical Attention for Nephropathy,N,NaN,100.0,59.0,NaN,NaN,REG,N


- it looks like some groups can belong to or be associated with multiple ACOs. Because of this, let's just remove ones with not ACO association at all. 

In [28]:
measures_inter = measures_inter[(measures_inter["ACO_PC_ID_1"].isnull() == False) | (measures_inter["ACO_PC_ID_2"].isnull() == False)]

In [39]:
measures_inter.isnull().sum()

org_name                        0
group_PAC_ID                    0
state                           0
ACO_PC_ID_1                     0
ACO_name_1                      0
ACO_PC_ID_2                 16231
ACO_name_2                  16231
measure_code                    0
measure_title                   0
inverse_measure                 0
attestation_value           16528
measure_performance_rate        0
denominator_count               0
star_value                  16135
five_star_benchmark         16135
collection_type                 0
from_PC_livesite                0
dtype: int64

We have a few columns to still figure out. We can already plan to replace ACO related columns with None, but we can probably do the same with the star_value and five star benchmark since we know that only certain measures qualified for this kind of judgment (refer to star_ratings_explained doc). However, the attestation column is interesting. After removing some data, the `attestation_value` column now consists of 100% missing values. Let's take a quick look back at the original measures dataset to make sure we aren't removing anything important. 

In [54]:
measures.attestation_value.value_counts()

Y    20493
Name: attestation_value, dtype: int64

Attestation means the act of validating something. So perhaps, this column represents whether the measure has been validated? It is tough to say. 

In [55]:
measures.collection_type.value_counts()

ATT     31140
REG     19871
WI       1336
QCDR      481
Name: collection_type, dtype: int64

There is a collection type called "**CMS Web Attestation**" and it makes up for 31,140 entries in the df. To be safe we will remove this column, but we may come back to it a later time.  

In [57]:
# Based on our exploratin, we will drop the attestation column and fill any remaining NaN values with "None"
measures_inter = measures_inter.drop(axis = 'columns',columns = ['attestation_value']).fillna(value = "None")

In [59]:
measures_inter.isnull().sum()

org_name                    0
group_PAC_ID                0
state                       0
ACO_PC_ID_1                 0
ACO_name_1                  0
ACO_PC_ID_2                 0
ACO_name_2                  0
measure_code                0
measure_title               0
inverse_measure             0
measure_performance_rate    0
denominator_count           0
star_value                  0
five_star_benchmark         0
collection_type             0
from_PC_livesite            0
dtype: int64

In [60]:
measures_inter.shape

(16528, 16)

The original shape was (73321, 17) and the cleaned shape is (16528, 16). We lost quite a lot of data. 

In [61]:
measures_inter.to_csv("data/cleaned_measures.csv")